Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [25]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [26]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [27]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [29]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

print('got here')

got here


Let's run this computation and iterate:

In [30]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 20.415340
Training accuracy: 11.0%
Validation accuracy: 12.8%
Loss at step 100: 2.323436
Training accuracy: 71.0%
Validation accuracy: 69.8%
Loss at step 200: 1.860098
Training accuracy: 74.8%
Validation accuracy: 72.8%
Loss at step 300: 1.610386
Training accuracy: 76.2%
Validation accuracy: 73.8%
Loss at step 400: 1.442542
Training accuracy: 77.0%
Validation accuracy: 74.2%
Loss at step 500: 1.320233
Training accuracy: 77.6%
Validation accuracy: 74.4%
Loss at step 600: 1.225521
Training accuracy: 78.2%
Validation accuracy: 74.6%
Loss at step 700: 1.148918
Training accuracy: 78.8%
Validation accuracy: 74.8%
Loss at step 800: 1.085106
Training accuracy: 79.1%
Validation accuracy: 75.0%
Test accuracy: 82.6%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [47]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
print('got here')

got here


Let's run it:

In [32]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.784462
Minibatch accuracy: 3.9%
Validation accuracy: 5.8%
Minibatch loss at step 500: 1.148882
Minibatch accuracy: 81.2%
Validation accuracy: 75.2%
Minibatch loss at step 1000: 1.359468
Minibatch accuracy: 81.2%
Validation accuracy: 76.4%
Minibatch loss at step 1500: 0.813572
Minibatch accuracy: 82.8%
Validation accuracy: 77.5%
Minibatch loss at step 2000: 0.844316
Minibatch accuracy: 82.0%
Validation accuracy: 77.5%
Minibatch loss at step 2500: 1.049758
Minibatch accuracy: 76.6%
Validation accuracy: 78.5%
Minibatch loss at step 3000: 1.045930
Minibatch accuracy: 76.6%
Validation accuracy: 78.7%
Test accuracy: 86.5%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [134]:
# Define 1-hidden layer neural network with rectified linear units nn.relu() and 1024 hidden nodes


def forward_prop(dataset, weights1, biases1, weights2, biases2):
  calc1_output = tf.matmul(dataset, weights1) + biases1
  relu_output = tf.nn.relu(calc1_output)
  return tf.matmul(relu_output, weights2) + biases2


batch_size = 128

graph2 = tf.Graph()
with graph2.as_default():

  # Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.as_dtype(train_dataset.dtype), shape = (batch_size, train_dataset.shape[1]))
  tf_train_labels = tf.placeholder(tf.as_dtype(train_labels.dtype), shape = (batch_size, train_labels.shape[1]))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_size = train_dataset.shape[1]
  hidden_size = 1028
  output_size = train_labels.shape[1]
  weights1 = tf.Variable(tf.truncated_normal([input_size, hidden_size]))
#   biases1 = tf.Variable(tf.zeros([hidden_size]))
  biases1 = tf.Variable(tf.truncated_normal([hidden_size], stddev=1.0))
  weights2 = tf.Variable(tf.truncated_normal([hidden_size, output_size]))
#   biases2 = tf.Variable(tf.zeros([output_size]))
  biases2 = tf.Variable(tf.truncated_normal([output_size], stddev=1.0))
  
  # Decay rate
#   global_step = tf.Variable(0)
#   learning_rate = tf.train.exponential_decay(
#     0.2,                        # Base learning rate.
#     global_step * batch_size,   # Current index into the dataset.
#     1000,                       # Decay step.
#     0.96,                        # Decay rate.
#     staircase=True)
#   optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Training computation.
  logits = forward_prop(tf_train_dataset, weights1, biases1, weights2, biases2)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, weights1, biases1, weights2, biases2))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, weights1, biases1, weights2, biases2))
    
print('got here2')

got here2


In [135]:
# Run 1-hidden layer neural network with rectified linear units nn.relu() and 1024 hidden nodes

num_training_examples = train_dataset.shape[0]

with tf.Session(graph=graph2) as session:
  tf.initialize_all_variables().run()
  
  for step in range(3001):
    offset = (step * batch_size) % (num_training_examples - batch_size)

    batch_dataset = train_dataset[offset:(offset+batch_size), :]
    batch_labels = train_labels[offset:(offset+batch_size), :]

    feed_dict = {tf_train_dataset: batch_dataset, tf_train_labels: batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if step % 500 == 0:
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels), "\n")

  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Minibatch loss at step 0: 297.172424
Minibatch accuracy: 13.3%
Validation accuracy: 25.3% 

Minibatch loss at step 500: 20.524803
Minibatch accuracy: 84.4%
Validation accuracy: 81.5% 

Minibatch loss at step 1000: 16.297840
Minibatch accuracy: 78.9%
Validation accuracy: 82.9% 

Minibatch loss at step 1500: 4.893043
Minibatch accuracy: 92.2%
Validation accuracy: 82.5% 

Minibatch loss at step 2000: 4.767471
Minibatch accuracy: 84.4%
Validation accuracy: 83.7% 

Minibatch loss at step 2500: 6.136764
Minibatch accuracy: 85.2%
Validation accuracy: 84.3% 

Minibatch loss at step 3000: 3.977015
Minibatch accuracy: 83.6%
Validation accuracy: 83.4% 

Test accuracy: 90.2%


## 1-layer network

Minibatch loss at step %d: %f 0 17.5934
Minibatch accuracy: %.1f%% 11.71875
Validation accuracy: %.1f%% 11.51

Minibatch loss at step %d: %f 500 1.17863
Minibatch accuracy: %.1f%% 79.6875
Validation accuracy: %.1f%% 75.3

Minibatch loss at step %d: %f 1000 1.43065
Minibatch accuracy: %.1f%% 72.65625
Validation accuracy: %.1f%% 76.27

Minibatch loss at step %d: %f 1500 0.765105
Minibatch accuracy: %.1f%% 82.8125
Validation accuracy: %.1f%% 76.33

Minibatch loss at step %d: %f 2000 0.897331
Minibatch accuracy: %.1f%% 78.90625
Validation accuracy: %.1f%% 77.18

Minibatch loss at step %d: %f 2500 1.05324
Minibatch accuracy: %.1f%% 75.78125
Validation accuracy: %.1f%% 77.96

Minibatch loss at step %d: %f 3000 1.0329
Minibatch accuracy: %.1f%% 81.25
Validation accuracy: %.1f%% 78.66

Test accuracy: 86.0%




## 1-hidden layer neural network with rectified linear units
## learning rate 0.5

Minibatch loss at step %d: %f 0 472.167
Minibatch accuracy: %.1f%% 10.9375
Validation accuracy: %.1f%% 29.96 

Minibatch loss at step %d: %f 500 25.001
Minibatch accuracy: %.1f%% 76.5625
Validation accuracy: %.1f%% 79.47 

Minibatch loss at step %d: %f 1000 8.37329
Minibatch accuracy: %.1f%% 79.6875
Validation accuracy: %.1f%% 81.65 

Minibatch loss at step %d: %f 1500 6.6643
Minibatch accuracy: %.1f%% 87.5
Validation accuracy: %.1f%% 79.88 

Minibatch loss at step %d: %f 2000 3.36347
Minibatch accuracy: %.1f%% 87.5
Validation accuracy: %.1f%% 81.19 

Minibatch loss at step %d: %f 2500 2.70563
Minibatch accuracy: %.1f%% 84.375
Validation accuracy: %.1f%% 81.87 

Minibatch loss at step %d: %f 3000 2.23426
Minibatch accuracy: %.1f%% 79.6875
Validation accuracy: %.1f%% 81.88 

Test accuracy: 89.3%



## 1-hidden layer neural network with rectified linear units
## learning rate 0.2

Minibatch loss at step 0: 330.778046
Minibatch accuracy: 22.7%
Validation accuracy: 29.2% 

Minibatch loss at step 500: 12.663446
Minibatch accuracy: 85.9%
Validation accuracy: 81.0% 

Minibatch loss at step 1000: 10.323658
Minibatch accuracy: 79.7%
Validation accuracy: 82.5% 

Minibatch loss at step 1500: 8.173533
Minibatch accuracy: 89.8%
Validation accuracy: 83.5% 

Minibatch loss at step 2000: 5.301463
Minibatch accuracy: 91.4%
Validation accuracy: 83.6% 

Minibatch loss at step 2500: 5.601147
Minibatch accuracy: 85.2%
Validation accuracy: 83.7% 

Minibatch loss at step 3000: 4.422043
Minibatch accuracy: 79.7%
Validation accuracy: 84.3% 

Test accuracy: 91.2%




## 1-hidden layer neural network with rectified linear units
## learning rate 0.2 & AdagradOptimizer

Minibatch loss at step 0: 404.382202
Minibatch accuracy: 15.6%
Validation accuracy: 41.0% 

Minibatch loss at step 500: 11.391366
Minibatch accuracy: 84.4%
Validation accuracy: 81.3% 

Minibatch loss at step 1000: 11.932853
Minibatch accuracy: 76.6%
Validation accuracy: 81.6% 

Minibatch loss at step 1500: 7.554851
Minibatch accuracy: 89.8%
Validation accuracy: 82.6% 

Minibatch loss at step 2000: 5.627324
Minibatch accuracy: 84.4%
Validation accuracy: 83.0% 

Minibatch loss at step 2500: 5.328107
Minibatch accuracy: 87.5%
Validation accuracy: 83.9% 

Minibatch loss at step 3000: 3.800861
Minibatch accuracy: 87.5%
Validation accuracy: 83.5% 

Test accuracy: 90.7%